In [ ]:
%matplotlib inline
import numpy as np
import sklearn.datasets as data
import matplotlib.pyplot as plt
import matplotlib

import seaborn as sb; sb.set_style( 'darkgrid' ) # use whitegrid if prefer a white background
import pandas as pd

from numpy.random import SeedSequence, default_rng
rng = default_rng( SeedSequence().entropy )

import warnings
warnings.filterwarnings("ignore")

#matplotlib.rcParams.update( { 'font.size': 18 } ) # Use this to setup your preferred font size for plotting

In [ ]:
# Data
X, y = data.load_breast_cancer( return_X_y = True, as_frame = False )

In [ ]:
help( data.load_breast_cancer )

#### 1- Compare Gradient Boosting, SVM and Multi-layer Perceptrons using Cross validation with Receiver Characteristic Operator curves, and confusion matrix on the provided dataset above. That is,

+ Examine how regularization affects the training and cross-validation curves
+ Examine how different K-fold evaluations affects model fitting using the ROC curves
+ Select the best regulairzation parameter and then confusion matrices of the models 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

# Learning curves
from sklearn.model_selection import validation_curve

# Cross-validation and ROC curves
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold

In [ ]:
def plot_k_cv_roc( model, Kfolds, X, y, model_name ):
    '''
    Plots the ROC curves of each K-fold.
    model <- used estimator
    Kfolds <- Number of K-folds
    X <- feature data
    y <- label data
    model_name <- string containing model name for plot title
    '''
    K  = Kfolds
    cv = StratifiedKFold( n_splits = K )

    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    fig, ax = plt.subplots( figsize = ( 10, 8 ))
    for i, (train, test) in enumerate(cv.split(X, y)):
        model.fit(X[train], y[train])
        viz = RocCurveDisplay.from_estimator(model, X[test], y[test],
                             name='ROC fold {}'.format(i),
                             alpha=0.3, lw=1, ax=ax)
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

    ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
            label='Chance', alpha=.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(mean_fpr, mean_tpr, color='b',
            label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
            lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                    label=r'$\pm$ 1 std. dev.')

    ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
           title="{0} Receiver operating characteristic example".format( model_name ))
    plt.legend(bbox_to_anchor=(1,1), loc="upper left")

##### SVM: Using the Radial basis function $\text{ exp}(-\gamma||x_i - x_j||^2)$,  vary the $\gamma$ parameter which scales the size of the kernel. Also experiment with different regularization C terms.

In [ ]:
svc = SVC( C = 0.5, kernel = 'rbf' )

param_range = np.logspace(?, ?, ?)

train_scores, test_scores = validation_curve(
    svc, X, y, param_name="gamma", param_range=param_range,
    scoring="accuracy", n_jobs=4)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure( figsize = ( 10, 7 ) )
plt.title("Validation Curve with SVM")
plt.xlabel(r"Margin separation range")
plt.ylabel("Score")
plt.ylim(0.4, 1.2)
lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best");

In [ ]:
plot_k_cv_roc( svc, ?, X, y, 'SVM' )

###### MLP

In [ ]:
mlp = MLPClassifier( hidden_layer_sizes = (?, ?) ) # adding more hidden layers: ( a, b, c, ...)

param_range = np.logspace(?, ?, ?)

train_scores, test_scores = validation_curve(
    mlp, X, y, param_name="alpha", param_range=param_range,
    scoring="accuracy", n_jobs=4)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure( figsize = ( 10, 7 ) )
plt.title("Validation Curve with MLP")
plt.xlabel(r"Parameter range")
plt.ylabel("Score")
plt.ylim(0.4, 1.2)
lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best");

In [ ]:
plot_k_cv_roc( mlp, ?, X, y, 'MLP' )

###### Gradient boosting

In [ ]:
gb = GradientBoostingClassifier()

param_range = np.arange( ?, ? )

train_scores, test_scores = validation_curve(
    gb, X, y, param_name="n_estimators", param_range=param_range,
    scoring="accuracy", n_jobs=4)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure( figsize = ( 10, 7 ) )
plt.title("Validation Curve with Gradient boosting")
plt.xlabel(r"Parameter range")
plt.ylabel("Score")
plt.ylim(0.4, 1.2)
lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best");

In [ ]:
plot_k_cv_roc( gb, ?, X, y, 'Gradient boosting' )

#### Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

cm = confusion_matrix( y, mlp.predict( X ) )

cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
cm = confusion_matrix( y, svc.predict( X ) )

cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
cm = confusion_matrix( y, gb.predict( X ) )

cm_display = ConfusionMatrixDisplay(cm).plot()

#### 2- Use Grid search for searching hyperparamters for the Multi-layered Perceptron. Fix some other hyperparamters by guessing but use Grid search to select the best solver, activation function, and number of hidden layers
+ Bonus: Do the same with SVM and Gradient boosting if you have time

In [ ]:
X, y = data.load_digits( return_X_y = True )

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# You may choose a different test_size
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.5, random_state = rng.integers( 432 ) )

tuned_parameters = [{'solver': ['sgd', 'adam', 'lbfgs'], 
                     'activation': ['relu', 'logistic'], 
                     'hidden_layer_sizes': [ (10, ), ( 15, 15 ), (18, 10 )]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        MLPClassifier( max_iter = 5), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

#### 3- Use MLP, SVM or Gradient boosting to do a recursive feature selection on the breast cancer dataset.
+ Bonus: If time, use the approach in Task 2, then move perform a recursive feature elimination with the hyperparameter optimized model. Is this a valid approach?

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV # use help to check RFECV

X, y = data.load_breast_cancer( return_X_y = True, as_frame = True )
print( X.shape )
X.head()

In [ ]:
svc                 = SVC( kernel = 'linear')
Kfold               = 5
select_min_features = 6

rfecv = RFECV(estimator=svc, step = 1, 
              cv = StratifiedKFold( Kfold ),
              scoring='accuracy',
              min_features_to_select = 5 )
rfecv.fit(X, y)

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores

n_scores = len(rfecv.cv_results_["mean_test_score"])
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Mean test accuracy")
plt.errorbar(
    range(select_min_features, n_scores + select_min_features),
    rfecv.cv_results_["mean_test_score"],
    yerr=rfecv.cv_results_["std_test_score"],
)
plt.title("Recursive Feature Elimination")
plt.show();

#### 4- This an advance approach to model selection based on Bayesian Analysis. If you feel bored, then proceed to do this. Use whatever model you wish. This solution example is done using a MLP model.

#### Use the approach in Task 2 to hyperparameter optimize a selected model. Use either the same set of candidate parameters or make up more.

+ To understan the Bayes Analysis -based comparison of models, read on Region of Practical Equivalence https://doingbayesiandataanalysis.blogspot.com/2013/08/how-much-of-bayesian-posterior.html 

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

# recommend using a small number of candidates to make sense of some plots
tuned_parameters = [{'solver': ['sgd', 'adam', 'lbfgs'], 
                     'activation': ['relu', 'logistic'], 
                     'hidden_layer_sizes': [10, 15, 18]}]

cv = RepeatedStratifiedKFold(
    n_splits=2, n_repeats=2, random_state=rng.integers( 5324 )
)

mlp = MLPClassifier( max_iter = 5)

search = GridSearchCV( estimator = mlp, param_grid=tuned_parameters, scoring='roc_auc', cv=cv )
search.fit(X, y)

In [ ]:
results_df = pd.DataFrame(search.cv_results_)
results_df = results_df.sort_values(by=['rank_test_score'])
results_df = (
    results_df
    .set_index(results_df["params"].apply(
        lambda x: "_".join(str(val) for val in x.values()))
    )
    .rename_axis('kernel')
)
results_df[
    ['params', 'rank_test_score', 'mean_test_score', 'std_test_score']
]

In [ ]:
# create df of model scores ordered by perfomance
model_scores = results_df.filter(regex=r'split\d*_test_score')

# plot 30 examples of dependency between cv fold and AUC scores
fig, ax = plt.subplots()
sb.lineplot(
    data=model_scores.transpose().iloc[:30],
    dashes=False, palette='Set1', marker='o', alpha=.5, ax=ax
)
ax.set_xlabel("CV test fold", size=12, labelpad=10)
ax.set_ylabel("Model AUC", size=12)
ax.tick_params(bottom=True, labelbottom=False)
ax.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1)
#plt.legend(bbox_to_anchor=(1,1), loc="upper left")
plt.show()

# print correlation of AUC scores across folds
print(f"Correlation of models:\n {model_scores.transpose().corr()}")

In [ ]:
from scipy.stats import t


def corrected_std(differences, n_train, n_test):
    """Corrects standard deviation using Nadeau and Bengio's approach.

    Parameters
    ----------
    differences : ndarray of shape (n_samples, 1)
        Vector containing the differences in the score metrics of two models.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    corrected_std : int
        Variance-corrected standard deviation of the set of differences.
    """
    n = n_train + n_test
    corrected_var = (
        np.var(differences, ddof=1) * ((1 / n) + (n_test / n_train))
    )
    corrected_std = np.sqrt(corrected_var)
    return corrected_std


def compute_corrected_ttest(differences, df, n_train, n_test):
    """Computes right-tailed paired t-test with corrected variance.

    Parameters
    ----------
    differences : array-like of shape (n_samples, 1)
        Vector containing the differences in the score metrics of two models.
    df : int
        Degrees of freedom.
    n_train : int
        Number of samples in the training set.
    n_test : int
        Number of samples in the testing set.

    Returns
    -------
    t_stat : float
        Variance-corrected t-statistic.
    p_val : float
        Variance-corrected p-value.
    """
    mean = np.mean(differences)
    std = corrected_std(differences, n_train, n_test)
    t_stat = mean / std
    p_val = t.sf(np.abs(t_stat), df)  # right-tailed t-test
    return t_stat, p_val

In [ ]:
model_1_scores = model_scores.iloc[0].values  # scores of the best model
model_2_scores = model_scores.iloc[1].values  # scores of the second-best model

differences = model_1_scores - model_2_scores

n       = differences.shape[0]  # number of test sets
df      = n - 1
n_train = len(list(cv.split(X, y))[0][0])
n_test  = len(list(cv.split(X, y))[0][1])

# intitialize random variable
t_post = t(
    df, loc=np.mean(differences),
    scale=corrected_std(differences, n_train, n_test)
)

x = np.linspace(t_post.ppf(0.001), t_post.ppf(0.999), 100)

plt.plot(x, t_post.pdf(x))
plt.xticks(np.arange(-0.5, 0.01, 0.5))
plt.fill_between(x, t_post.pdf(x), 0, facecolor='blue', alpha=.2)
plt.ylabel("Probability density")
plt.xlabel(r"Mean difference ($\mu$)")
plt.title("Posterior distribution")
plt.show()

In [ ]:
better_prob = 1 - t_post.cdf(0)

print(f"Probability of {model_scores.index[0]} being more accurate than "
      f"{model_scores.index[1]}: {better_prob:.3f}")
print(f"Probability of {model_scores.index[1]} being more accurate than "
      f"{model_scores.index[0]}: {1 - better_prob:.3f}")

In [ ]:
rope_interval = [-0.01, 0.01]
rope_prob = t_post.cdf(rope_interval[1]) - t_post.cdf(rope_interval[0])

print(f"Probability of {model_scores.index[0]} and {model_scores.index[1]} "
      f"being practically equivalent: {rope_prob:.3f}")

In [ ]:
x_rope = np.linspace(rope_interval[0], rope_interval[1], 100)

plt.plot(x, t_post.pdf(x))
plt.xticks(np.arange(-0.5, 0.01, 0.5))
plt.vlines([-0.01, 0.01], ymin=0, ymax=(np.max(t_post.pdf(x)) + 1))
plt.fill_between(x_rope, t_post.pdf(x_rope), 0, facecolor='blue', alpha=.2)
plt.ylabel("Probability density")
plt.xlabel(r"Mean difference ($\mu$)")
plt.title("Posterior distribution under the ROPE")
plt.show()

In [ ]:
cred_intervals = []
intervals = [0.5, 0.75, 0.95]

for interval in intervals:
    cred_interval = list(t_post.interval(interval))
    cred_intervals.append([interval, cred_interval[0], cred_interval[1]])

cred_int_df = pd.DataFrame(
    cred_intervals,
    columns=['interval', 'lower value', 'upper value']
).set_index('interval')
cred_int_df

In [ ]:
from itertools import combinations
from math import factorial

n_comparisons = (
    factorial(len(model_scores))
    / (factorial(2) * factorial(len(model_scores) - 2))
)

pairwise_bayesian = []

for model_i, model_k in combinations(range(len(model_scores)), 2):
    model_i_scores = model_scores.iloc[model_i].values
    model_k_scores = model_scores.iloc[model_k].values
    differences = model_i_scores - model_k_scores
    t_post = t(
        df, loc=np.mean(differences),
        scale=corrected_std(differences, n_train, n_test)
    )
    worse_prob = t_post.cdf(rope_interval[0])
    better_prob = 1 - t_post.cdf(rope_interval[1])
    rope_prob = t_post.cdf(rope_interval[1]) - t_post.cdf(rope_interval[0])

    pairwise_bayesian.append([model_scores.index[model_i], model_scores.index[model_k], worse_prob, better_prob, rope_prob])

pairwise_bayesian_df = (pd.DataFrame(
    pairwise_bayesian,
    columns=['model_1', 'model_2','worse_prob', 'better_prob', 'rope_prob']
).round(3))

pairwise_bayesian_df